In [1]:
from pysgx import info, stocks, util
import importlib
import pandas as pd
import numpy as np

In [7]:
import warnings

warnings.filterwarnings('ignore')

In [70]:
# Re-run this cell if pysgx module have hot changes
importlib.reload(info);
importlib.reload(stocks);
importlib.reload(util);

# Portfolio

In [3]:
# Load ALL stocks!
all_stocks = stocks.loads(info.all_tickers)

def get_portfolio(tickers):
    portfolio = []
    for stock in all_stocks:
        if stock.ticker in tickers:
            portfolio.append(stock)
    return portfolio

# Rolling Test using AddBack Close Return

In [ ]:
period_start = util.YearMonth(2000, 1)
period_last = util.YearMonth(2023, 1)

ByAddBackClose = []
ByAdjClose = []

while period_start <= period_last:
    returns = dict()
    for stock in all_stocks:
        period_end = util.get_next(period_start, 12)
        adtv = stock.get_adtv(period_start, period_end)

        if adtv > 100000:
            r = stocks.get_return(period_start, period_end, stock, False)
            if not np.isnan(r.ByAddBackClose):
                returns[stock.ticker] = r.ByAddBackClose

    # sort to find top 10 return stocks using AddBack Close
    returns2 = sorted(returns.items(), key=lambda x: x[1], reverse=True)[:10]
    top_returns = {k[0]: returns[k[0]] for k in returns2}

    print("Evaluate 1Y since: ", period_start, top_returns.keys())
    eval_period_end = util.get_next(period_end, 3)
    if len(top_returns) == 10:
        r = stocks.get_portfolio_return(period_end, eval_period_end, get_portfolio(top_returns.keys()))
        print("backtest 1Q", period_end, eval_period_end, r)
        ByAdjClose.append(r.ByAdjClose)
        ByAddBackClose.append(r.ByAddBackClose)
    print("")

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

In [86]:
df = pd.DataFrame(ByAddBackClose, columns=['ByAddBackClose'])
returns = df['ByAddBackClose'] + 1
returns.product()

3.07166240833986

In [85]:
df = pd.DataFrame(ByAdjClose, columns=['ByAdjClose'])
returns = df['ByAdjClose'].dropna() + 1
returns.product()

6.466738845153519

# Rolling Test using Adj Close Return

In [ ]:
period_start = util.YearMonth(2000, 1)
period_last = util.YearMonth(2023, 1)

ByAddBackClose = []
ByAdjClose = []

while period_start <= period_last:
    returns = dict()
    for stock in all_stocks:
        period_end = util.get_next(period_start, 12)
        adtv = stock.get_adtv(period_start, period_end)

        if adtv > 100000:
            r = stocks.get_return(period_start, period_end, stock, False)
            if not np.isnan(r.ByAdjClose):
                returns[stock.ticker] = r.ByAdjClose

    # sort to find top 10 return stocks using Adj Close
    returns2 = sorted(returns.items(), key=lambda x: x[1], reverse=True)[:10]
    top_returns = {k[0]: returns[k[0]] for k in returns2}

    print("Evaluate 1Y since: ", period_start, top_returns.keys())
    eval_period_end = util.get_next(period_end, 3)
    if len(top_returns) == 10:
        r = stocks.get_portfolio_return(period_end, eval_period_end, get_portfolio(top_returns.keys()))
        print("backtest 1Q", period_end, eval_period_end, r)
        ByAdjClose.append(r.ByAdjClose)
        ByAddBackClose.append(r.ByAddBackClose)
    print("")

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

In [78]:
df = pd.DataFrame(ByAddBackClose, columns=['ByAddBackClose'])
returns = df['ByAddBackClose'] + 1
returns.product()

1.8382072607663853

In [82]:
df = pd.DataFrame(ByAdjClose, columns=['ByAdjClose'])
returns = df['ByAdjClose'].dropna() + 1
returns.product()

98.17057979089792

# Manual portfolio

In [89]:
dy1 = ["M01", "5DD", "CC3", "i07", "BN4"]
dy2 = ["L38", "E5H", "1D0", "Z74", "CY6U"]
ww = ["A04", "5VP", "BQD", "F03", "NO4", "41B", "A34", "T14", "Q0X"]

period_start = util.YearMonth(2015, 1)
period_last = util.YearMonth(2023, 1)

portfolio = stocks.loads(ww)
while period_start <= period_last:
    period_end = util.get_next(period_start, 12)
    backtest_period_start = period_end
    backtest_period_end = util.get_next(backtest_period_start, 3)

    returns = stocks.get_returns(period_start, period_end, portfolio)
    w1 = stocks.get_weights_by_returns(returns)

    r = stocks.get_portfolio_return(backtest_period_start, backtest_period_end, portfolio)
    print(backtest_period_start, backtest_period_end, r)

    r = stocks.get_portfolio_return(period_end, eval_period_end, portfolio, w1.ByAddBackClose)
    print(backtest_period_start, backtest_period_end, r)

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

2016-01-01 2016-04-01 ByAdjClose: -0.061563 ByAddBackClose: -0.061487
2016-01-01 2016-04-01 ByAdjClose:  1.226394 ByAddBackClose:  1.439357
2016-04-01 2016-07-01 ByAdjClose: -0.009272 ByAddBackClose: -0.012249
2016-04-01 2016-07-01 ByAdjClose:  1.147053 ByAddBackClose:  1.065803
2016-07-01 2016-10-01 ByAdjClose: -0.023897 ByAddBackClose: -0.023897
2016-07-01 2016-10-01 ByAdjClose:  0.894608 ByAddBackClose:  0.672889
2016-10-01 2017-01-01 ByAdjClose:  0.132548 ByAddBackClose:  0.129483
2016-10-01 2017-01-01 ByAdjClose: -0.387942 ByAddBackClose: -0.522571
2017-01-01 2017-04-01 ByAdjClose:  0.139831 ByAddBackClose:  0.139806
2017-01-01 2017-04-01 ByAdjClose:  27.923594 ByAddBackClose:  26.278453
2017-04-01 2017-07-01 ByAdjClose:  0.023692 ByAddBackClose:  0.021239
2017-04-01 2017-07-01 ByAdjClose:  1.846036 ByAddBackClose:  1.674333
2017-07-01 2017-10-01 ByAdjClose: -0.013235 ByAddBackClose: -0.013235
2017-07-01 2017-10-01 ByAdjClose:  1.032140 ByAddBackClose:  0.938598
2017-10-01 2018-01

In [ ]:
# stock = stocks.load("D05")
# print(stock.get_sector())
df = util.to_df(info.get_all_avg_vol(), ['Ticker', 'AvergeVolume'])
df.AvergeVolume = df.AvergeVolume / 10000
df = df[df.AvergeVolume>1].sort_values(by=['AvergeVolume'])

# df = util.to_df(info.get_all_mc(), ['Ticker', 'MarketCap'])
# df.MarketCap = df.MarketCap / 1000000
# df = df[df.MarketCap>1000].sort_values(by=['MarketCap'])

items = stocks.loads(df.Ticker.to_list())

for stock in items:
    print(stock)

# Adj Close return vs AddBack Return

In [183]:
# stock = stocks.load("D05")
stock = stocks.load("D8DU")
df = stocks.get_addback_close(stock, "2022-01-01", "2024-01-01")

return_adj_close = df["Adj Close"].pct_change().dropna()
return_addback_close = df["AddBackClose"].pct_change().dropna()

print(return_addback_close.mean())
print(return_adj_close.mean())
return_adj_close.std()  / return_addback_close.std() - 1
# return_adj_close.mean()  / return_addback_close.mean() - 1

-0.0002463114195503293
-0.00044110314428133615


0.04505055363062538

In [195]:
df = pd.concat([return_addback_close, return_adj_close], axis=1)
df.cov().iloc[0, 1]

0.000718800466679042

# Stock Market Cap Distribution

In [ ]:
df = util.to_df(info.get_all_mc(), ['Ticker', 'MarketCap'])
df.MarketCap = df.MarketCap / 1000000
plt = df[df.MarketCap<100].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - Below(100M SGD)')
plt.legend(['Total 365 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_100M.png')

plt = df[df.MarketCap>100][df.MarketCap<1500].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - 100M~1.5B SGD)')
plt.legend(['Total 188 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_100M_1.5B.png')

plt = df[df.MarketCap>1500][df.MarketCap<30000].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - 1.5B~30B SGD)')
plt.legend(['Total 62 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_1.5B_20B.png')

df[df.MarketCap>10000].sort_values(by=['MarketCap']).to_csv("mc_10B_plus.csv", index=False, float_format='%.6f')

# Average Daily Trading Volume Hist

In [ ]:
df = util.to_df(info.get_all_avg_vol(), ['Ticker', 'AvergeVolume'])
df.AvergeVolume = df.AvergeVolume / 10000

plt = df[df.AvergeVolume<=1].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) - Below(10K SGD)')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 365 stocks'])
plt.get_figure().savefig('adtv_10K.png')

plt = df[1<df.AvergeVolume][df.AvergeVolume<=30].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) 10K~300K SGD')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 207 stocks'])
plt.get_figure().savefig('adtv_10K_300K.png')

plt = df[df.AvergeVolume>30][df.AvergeVolume<=300].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) 300K~3M SGD')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 68 stocks'])
plt.get_figure().savefig('adtv_300K_3M.png')

df[df.AvergeVolume>300].sort_values(by=['AvergeVolume']).to_csv("adtv_3M_plus.csv", index=False, float_format='%.6f')

# STI Estimate

In [ ]:
es3 = stocks.load("ES3")
df = stocks.get_addback_close(es3, "2010-03-01", "2024-03-15")
df["Adj Close"] = df["Adj Close"] * 990
df["AddBackClose"] = df["AddBackClose"] * 1010
df["Close"] = df["Close"] * 1000

plt = df.plot(x="Day", y = ["AddBackClose", "Close", "Adj Close"], linewidth=0.8, figsize=(15, 10), title="Esitmated STI")
plt.legend(['Estimated STI if add back dividend', 'STI', 'Estimated STI if use Adj Close'])
plt.get_figure().savefig('STI_estimated.png')